In [14]:
import json
import pandas as pd
import numpy as np
import csv
import os
import ast

import nltk
import gensim

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from wordcloud import WordCloud

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sebre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sebre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sebre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
# test classification dataset
from collections import Counter
from sklearn.datasets import make_classification

# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

In [4]:
firehouse = pd.read_csv('firehouse_with_reviews.csv', sep = ',')
subway = pd.read_csv('subway_with_reviews.csv', sep = ',')
jmike = pd.read_csv('jersey_mike_with_reviews.csv', sep = ',')
jj = pd.read_csv('jimmy_johns_with_reviews.csv', sep = ',')
potbelly = pd.read_csv('potbelly_with_reviews.csv', sep = ',')
quiznos = pd.read_csv('quiznos_with_reviews.csv', sep = ',')

In [7]:
firehouse.head()
subway.head()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,0,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",WBq2S1dhTme48WSYtk-Mkg,fVup-p6mgIMuWzNIj-CS1Q,1,0,0,0,"Do not eat here. First of all, it took them ab...",12/4/2016 22:28
1,1,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",f-0P4HgQvtFR4OQgT-kFqQ,Lw6rx2yyzUbnFMUiD-Mk3Q,5,1,0,0,"Great food, clean establishment, and quick ser...",4/22/2017 6:24
2,2,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",8OCbRtYMBu3sm8jZXMQxiA,Fh4gvyRoRjtYBhdTL6dDBg,2,1,0,0,Been standing here 15 min now. Three people ah...,3/23/2014 22:19
3,3,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",89u5jcXM5M9zKLxzhgILkQ,KXDiP5LvmOIAH2M7rQyquw,2,0,0,0,Went over to Subway last evening their were 2 ...,1/11/2016 8:33
4,4,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",Co8Kuvrsb_tiw4COwt3Izw,fVup-p6mgIMuWzNIj-CS1Q,1,0,0,0,Tried it again. Big mistake. I thought 2017 mi...,3/6/2017 22:30


In [10]:
frames = [firehouse, subway, jmike, jj, potbelly, quiznos]
df = pd.concat(frames)

In [16]:
df.tail()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
319,319,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",pfeVcCj_HOe9bK3nN5K82g,Icoqtxs4WZGdWYxhkSjovQ,5,0,0,0,From what I know this is family owned and oper...,2018-11-03 18:16:24
320,320,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",CRwkpCKg8q3yN8B3sSdFSA,x8_WrbvIgqZ8-5BtJofc2Q,2,0,2,0,I use to go to the Quiznos in st. Charles on a...,2017-09-02 01:13:52
321,321,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",OkvyYXHhjqUWX1pAmobSxw,_V4EtVZ5KlCFIZ4U6nN7XQ,5,0,0,0,If you want the best sandwich experience you'v...,2020-02-06 21:45:19
322,322,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",5A-7dzFGxCUHeH5st8XEYg,TESUeCPNTpoovu7sJypdzw,5,1,0,0,"Oh Quizno's, how I missed you.\nThis restauran...",2013-05-16 00:31:16
323,323,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",jPE_eLIoe3huZuQIGrYyaA,CQDg6a4wDgkqRUJZSzXc-w,5,1,0,0,I absolutely LOVE Quiznos. I will drive 30 min...,2019-07-02 06:24:22


In [17]:
# merge reviews texts by business_id in df_merge
df_merge = df.groupby(['business_id'], as_index=False).first()

In [18]:
df_merge

,business_id,Unnamed: 0,name,address,city,state,postal_code,latitude,longitude,stars_x,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,#NAME?,336,Subway,1 Brookings Dr,Saint Louis,MO,63105,38.647482,-90.309581,2.0,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",MYctGuflZVL9ctEF1--9qQ,0MHq2xX2O2WUbbW2BMwyhw,1,0,0,0,I repeated my order three times loudly and you...,11/21/2017 1:15
1,-2CPhK6ik9ZBgFX_F-dkxQ,3255,Subway,"14961 N Florida Ave, Ste 14961",Tampa,FL,33613,28.087029,-82.458310,2.0,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '8:0-21:30', ...",I6svHqxWhJnkQDkTKUPvbA,3DbINcPH5PI0DGEQp5mwbA,4,0,0,0,Just your typical subway. This one is particul...,10/24/2013 17:38
2,-7UDKbg_8TL4LVuYR6Ooyw,2953,Subway,"2451 Mcmullen Booth Rd, Ste 1",Clearwater,FL,33759,28.010475,-82.709550,3.0,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '9:0-22:0', '...",vI40HCh41z4-m5xI5Eog_w,V7-EP1CbhjQeEq7f6ab3DQ,4,1,0,1,Went here tonight and had a very nice experien...,2/28/2015 2:48
3,-bcnY0PELKRomT2z7eGQ3w,2048,Jersey Mike's Subs,575 Horsham Rd,Horsham,PA,19044,40.187752,-75.147294,3.5,...,"Delis, Sandwiches, Restaurants, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",PuDhaY19Jm4nIjcUh1GzBw,RlX07vMSZNsIhx2WOcO6wQ,5,0,1,0,One bite and I'm hooked! Friendly energetic st...,2016-10-01 17:27:49
4,-dEuJqTHuE6rv8MrndqoXA,1784,Jimmy John's,659 W Brandon Blvd,Brandon,FL,33511,27.937170,-82.293415,3.0,...,"Sandwiches, Food, Restaurants, Fast Food, Food...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",W6z_xkFWpBs3cIR7x_qdgQ,CsknEPODx4mjx1GQBUkMsw,4,0,0,0,"Pretty good, their bread is kinda hit or miss,...",2016-10-26 17:17:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,zQB2BH0NRx8r8kvDQXZ8mQ,2718,Jimmy John's,40715 US Hwy 19 N,Tarpon Springs,FL,34689,28.142290,-82.739463,4.0,...,"Fast Food, Food Delivery Services, Food, Sandw...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",pl5KBp9RCRPm45RYMkQCIw,Um5bfs5DH6eizgjH3xZsvg,3,0,0,0,"My first time into a Jimmy Johns, and I'm glad...",2014-08-10 20:38:49
830,zRjpagNQNZoovYfnmKxYmw,1085,Subway,"1084 E Lancaster Ave, Ste 1",Bryn Mawr,PA,19010,40.026373,-75.328650,3.0,...,"Restaurants, Fast Food, Sandwiches","{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",XKDQ-2ex4mBNGgxf7xAhfg,eFUcc2QbaW0InWgsBsMiwA,3,1,0,0,I've gotten a few sandwiches from here. It's ...,8/4/2016 14:02
831,zciO1XFzcKlX98Bi3XOGww,625,Subway,"3901 N Belt W, Ste B",Bellville,IL,62226,38.541845,-90.013896,1.5,...,"Restaurants, Sandwiches, Fast Food","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",XgFyQatlwTUj2y4nDTDk2g,#NAME?,1,0,0,0,HORRIBLE!! GO TO ANOTHER SUBWAY NOT THIS ONE!...,2/25/2019 23:26
832,zqsmGvH1aPMDFJB-jFNPwQ,3689,Jimmy John's,15069 Manchester Rd,Ballwin,MO,63011,38.593303,-90.549531,1.5,...,"Delis, Fast Food, Restaurants, Food, Sandwiche...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",aXTdF9clU0cTqgAEKTt8KA,hGJ7kvBRdiiUscHsirlxbQ,1,0,0,0,After 30 minutes I had no food. Called up ther...,2017-11-18 21:51:42


In [26]:
df_review = df_merge[['business_id', 'name', 'address', 'stars_x', 'text']]
df_review

,business_id,name,address,stars_x,text
0,#NAME?,Subway,1 Brookings Dr,2.0,I repeated my order three times loudly and you...
1,-2CPhK6ik9ZBgFX_F-dkxQ,Subway,"14961 N Florida Ave, Ste 14961",2.0,Just your typical subway. This one is particul...
2,-7UDKbg_8TL4LVuYR6Ooyw,Subway,"2451 Mcmullen Booth Rd, Ste 1",3.0,Went here tonight and had a very nice experien...
3,-bcnY0PELKRomT2z7eGQ3w,Jersey Mike's Subs,575 Horsham Rd,3.5,One bite and I'm hooked! Friendly energetic st...
4,-dEuJqTHuE6rv8MrndqoXA,Jimmy John's,659 W Brandon Blvd,3.0,"Pretty good, their bread is kinda hit or miss,..."
...,...,...,...,...,...
829,zQB2BH0NRx8r8kvDQXZ8mQ,Jimmy John's,40715 US Hwy 19 N,4.0,"My first time into a Jimmy Johns, and I'm glad..."
830,zRjpagNQNZoovYfnmKxYmw,Subway,"1084 E Lancaster Ave, Ste 1",3.0,I've gotten a few sandwiches from here. It's ...
831,zciO1XFzcKlX98Bi3XOGww,Subway,"3901 N Belt W, Ste B",1.5,HORRIBLE!! GO TO ANOTHER SUBWAY NOT THIS ONE!...
832,zqsmGvH1aPMDFJB-jFNPwQ,Jimmy John's,15069 Manchester Rd,1.5,After 30 minutes I had no food. Called up ther...


## NLP Text Clean

In [44]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")

In [134]:
len(stopwords)

179

In [135]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [45]:
# Load the regular expression library
import re

# Remove punctuation
df_review['text_processed'] = \
df_review['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_review['text_processed'] = \
df_review['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
df_review['text_processed'].tail()

829    my first time into a jimmy johns and i'm glad ...
830    i've gotten a few sandwiches from here  it's p...
831    horrible  go to another subway not this one  i...
832    after 30 minutes i had no food called up there...
833    this place rocks love them subs get over there...
Name: text_processed, dtype: object

In [46]:
# remove stopwords
# DONT REMOVE STOPWORDS FOR BIGRAMS
def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in stopwords]
    return (" ").join(tokens_filtered)

def stopwords_list(df):
    res = []
    for sent in df['text_processed']:
        res.append(remove_mystopwords(sent))
    return res

In [50]:
# stop_removed = stopwords_list(df_merge)

In [51]:
# stop_removed

["repeated order three times loudly still got wrong that's fine sudden became rude aggressive pointed excuse besides pretty good job telling exactly opposite everyone else behind glass keeping rolling eyes me\n\nthis level customer service eye-opening",
 'typical subway one particularly close home convenient need quick dinner want skip greasy fast food alot options people watching weight also quite opposite ive never issues food customer service ill keep coming long stays way',
 'went tonight nice experience mark sandwich artist made us couple good subs efficient wanted tip learn owner decided allow hope paying reasonable rate mark deserves',
 "one bite i'm hooked friendly energetic staff made pleasure soon tasted sub (turkey provolone 'mike's way' hot peppers) knew found one favorite local places eat",
 'pretty good bread kinda hit miss ive gotten 5-6 sandwiches last months kinda stale bread :/ cheap though guess cant complain much\n\nplus bring us free party platers sandwiches someti

In [53]:
# tokenization and stemming/lemmatizing

from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()


def tokenization_and_lemmatizing(text):
    
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    # stems = [stemmer.stem(t) for t in filtered_tokens]
    stems = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    return stems

def list_tokenize_lemm(lst):
    res = []
    for sent in lst:
        res.append(tokenization_and_lemmatizing(sent))
    return res

In [54]:
processed_txt = list_tokenize_lemm(stop_removed)

In [55]:
processed_txt

[['repeated',
  'order',
  'three',
  'time',
  'loudly',
  'still',
  'got',
  'wrong',
  'fine',
  'sudden',
  'became',
  'rude',
  'aggressive',
  'pointed',
  'excuse',
  'besides',
  'pretty',
  'good',
  'job',
  'telling',
  'exactly',
  'opposite',
  'everyone',
  'else',
  'behind',
  'glass',
  'keeping',
  'rolling',
  'eye',
  'level',
  'customer',
  'service'],
 ['typical',
  'subway',
  'one',
  'particularly',
  'close',
  'home',
  'convenient',
  'need',
  'quick',
  'dinner',
  'want',
  'skip',
  'greasy',
  'fast',
  'food',
  'alot',
  'option',
  'people',
  'watching',
  'weight',
  'also',
  'quite',
  'opposite',
  'ive',
  'never',
  'issue',
  'food',
  'customer',
  'service',
  'ill',
  'keep',
  'coming',
  'long',
  'stay',
  'way'],
 ['went',
  'tonight',
  'nice',
  'experience',
  'mark',
  'sandwich',
  'artist',
  'made',
  'u',
  'couple',
  'good',
  'sub',
  'efficient',
  'wanted',
  'tip',
  'learn',
  'owner',
  'decided',
  'allow',
  'hope'

## TD-IDF Vectorizer

Example code for using the sklearn module:

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?'
]
# corpus = pd.DataFrame(corpus)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
# array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
#        'this'], ...)
print(X.shape)

(4, 9)


(4, 9)

Our punctuation-removed corpus:

In [72]:
vectorizer = TfidfVectorizer(ngram_range = (1, 2))
X = vectorizer.fit_transform(stop_removed)
vectorizer.get_feature_names_out()
# array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
#        'this'], ...)
print(X.shape)

(834, 31605)


In [75]:
# USE THIS CODE FOR DATA FRAME
vect = TfidfVectorizer(ngram_range = (1, 2))
tfidf_matrix = vect.fit_transform(stop_removed)
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names_out())

In [83]:
df

,00,00 bars,00 pm,00 typically,00 went,000,000 sale,02,02 sit,05,...,zero customer,zero doritos,zero stars,zero tomatoes,zionsville,zip,zip around,zip right,zone,zone would
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
vect.get_feature_names_out()

array(['00', '00 bars', '00 pm', ..., 'zip right', 'zone', 'zone would'],
      dtype=object)

## N-Grams

In [23]:
from nltk import ngrams

# sentence = input("Enter the sentence: ")
# n = int(input("Enter the value of n: "))
lst_grams = []
sentence = 'My husband and I ordered sandwiches for pickup today, which we do a couple times a month. His pastrami was well-made with a good portion of deli meat. My tuna... well at first I thought they entirely forgot the tuna! Pulled it apart to find a tiny spoonful on each half-- absolutely laughable amount of filling! I order this sandwich regularly and it\'s NEVER like that. He took it back and asked for a remake,l. The assistant managers said "it\'s our portion size", and never apologized or offered either a refund or a remake. So $7 for a teaspoon of tuna, and apathetic response from the employees... you ARE KIDDING ME!! Customer service standards and quality need an overhaul in this location.'
n_grams = ngrams(sentence.split(), 2)
for grams in n_grams:
    lst_bigrams.append(grams)

In [25]:
n_grams

<generator object ngrams at 0x000002905BB32548>

With review sentences:

In [65]:
corpus = ','.join(stop_removed)

In [66]:
corpus

'repeated order three times loudly still got wrong that\'s fine sudden became rude aggressive pointed excuse besides pretty good job telling exactly opposite everyone else behind glass keeping rolling eyes me\n\nthis level customer service eye-opening,typical subway one particularly close home convenient need quick dinner want skip greasy fast food alot options people watching weight also quite opposite ive never issues food customer service ill keep coming long stays way,went tonight nice experience mark sandwich artist made us couple good subs efficient wanted tip learn owner decided allow hope paying reasonable rate mark deserves,one bite i\'m hooked friendly energetic staff made pleasure soon tasted sub (turkey provolone \'mike\'s way\' hot peppers) knew found one favorite local places eat,pretty good bread kinda hit miss ive gotten 5-6 sandwiches last months kinda stale bread :/ cheap though guess cant complain much\n\nplus bring us free party platers sandwiches sometimes,nothing 

In [70]:
from nltk import ngrams
lst_bigrams = []
n_grams = ngrams(corpus.split(), 2)
for grams in n_grams:
    lst_bigrams.append(grams)

In [71]:
lst_bigrams

[('repeated', 'order'),
 ('order', 'three'),
 ('three', 'times'),
 ('times', 'loudly'),
 ('loudly', 'still'),
 ('still', 'got'),
 ('got', 'wrong'),
 ('wrong', "that's"),
 ("that's", 'fine'),
 ('fine', 'sudden'),
 ('sudden', 'became'),
 ('became', 'rude'),
 ('rude', 'aggressive'),
 ('aggressive', 'pointed'),
 ('pointed', 'excuse'),
 ('excuse', 'besides'),
 ('besides', 'pretty'),
 ('pretty', 'good'),
 ('good', 'job'),
 ('job', 'telling'),
 ('telling', 'exactly'),
 ('exactly', 'opposite'),
 ('opposite', 'everyone'),
 ('everyone', 'else'),
 ('else', 'behind'),
 ('behind', 'glass'),
 ('glass', 'keeping'),
 ('keeping', 'rolling'),
 ('rolling', 'eyes'),
 ('eyes', 'me'),
 ('me', 'this'),
 ('this', 'level'),
 ('level', 'customer'),
 ('customer', 'service'),
 ('service', 'eye-opening,typical'),
 ('eye-opening,typical', 'subway'),
 ('subway', 'one'),
 ('one', 'particularly'),
 ('particularly', 'close'),
 ('close', 'home'),
 ('home', 'convenient'),
 ('convenient', 'need'),
 ('need', 'quick'),
 ('q

In [88]:
bigram_frequency = {}
for char in lst_bigrams:
    if char in bigram_frequency:
        bigram_frequency[char] += 1
    else:
        bigram_frequency[char] = 1
print(bigram_frequency)

{('repeated', 'order'): 1, ('order', 'three'): 1, ('three', 'times'): 3, ('times', 'loudly'): 1, ('loudly', 'still'): 1, ('still', 'got'): 1, ('got', 'wrong'): 1, ('wrong', "that's"): 1, ("that's", 'fine'): 2, ('fine', 'sudden'): 1, ('sudden', 'became'): 1, ('became', 'rude'): 1, ('rude', 'aggressive'): 1, ('aggressive', 'pointed'): 1, ('pointed', 'excuse'): 1, ('excuse', 'besides'): 1, ('besides', 'pretty'): 1, ('pretty', 'good'): 8, ('good', 'job'): 1, ('job', 'telling'): 1, ('telling', 'exactly'): 1, ('exactly', 'opposite'): 1, ('opposite', 'everyone'): 1, ('everyone', 'else'): 2, ('else', 'behind'): 1, ('behind', 'glass'): 1, ('glass', 'keeping'): 1, ('keeping', 'rolling'): 1, ('rolling', 'eyes'): 2, ('eyes', 'me'): 1, ('me', 'this'): 1, ('this', 'level'): 1, ('level', 'customer'): 1, ('customer', 'service'): 53, ('service', 'eye-opening,typical'): 1, ('eye-opening,typical', 'subway'): 1, ('subway', 'one'): 14, ('one', 'particularly'): 1, ('particularly', 'close'): 1, ('close', 'ho

In [90]:
# sort by descending count
bigram_frequency_sorted = sorted(bigram_frequency.items(), 
                              key = lambda kv: kv[1], 
                              reverse = True)
type(bigram_frequency_sorted)

list

In [96]:
# save to csv upload to github
with open('bigram_counts.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    # for key, value in enumerate(bigram_frequency_sorted):
    writer.writerow(fields)

NameError: name 'fields' is not defined

In [97]:
bigram_frequency_sorted.to_csv('bigram_counts.csv') 

AttributeError: 'list' object has no attribute 'to_csv'

In [100]:
bigram_df = pd.DataFrame(bigram_frequency_sorted)
bigram_df.columns = ['bigrams', 'counts']

In [101]:
bigram_df.to_csv('bigram_counts.csv')

## Linear Regression

In [108]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [102]:
# merge tf-idf and business data
# df = tf-idf matrix
pd.concat([df_review, df], axis=1)

,business_id,name,address,stars_x,text,00,00 bars,00 pm,00 typically,00 went,...,zero customer,zero doritos,zero stars,zero tomatoes,zionsville,zip,zip around,zip right,zone,zone would
0,#NAME?,Subway,1 Brookings Dr,2.0,I repeated my order three times loudly and you...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2CPhK6ik9ZBgFX_F-dkxQ,Subway,"14961 N Florida Ave, Ste 14961",2.0,Just your typical subway. This one is particul...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-7UDKbg_8TL4LVuYR6Ooyw,Subway,"2451 Mcmullen Booth Rd, Ste 1",3.0,Went here tonight and had a very nice experien...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-bcnY0PELKRomT2z7eGQ3w,Jersey Mike's Subs,575 Horsham Rd,3.5,One bite and I'm hooked! Friendly energetic st...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-dEuJqTHuE6rv8MrndqoXA,Jimmy John's,659 W Brandon Blvd,3.0,"Pretty good, their bread is kinda hit or miss,...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,zQB2BH0NRx8r8kvDQXZ8mQ,Jimmy John's,40715 US Hwy 19 N,4.0,"My first time into a Jimmy Johns, and I'm glad...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
830,zRjpagNQNZoovYfnmKxYmw,Subway,"1084 E Lancaster Ave, Ste 1",3.0,I've gotten a few sandwiches from here. It's ...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
831,zciO1XFzcKlX98Bi3XOGww,Subway,"3901 N Belt W, Ste B",1.5,HORRIBLE!! GO TO ANOTHER SUBWAY NOT THIS ONE!...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
832,zqsmGvH1aPMDFJB-jFNPwQ,Jimmy John's,15069 Manchester Rd,1.5,After 30 minutes I had no food. Called up ther...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
df_no0 = df.drop(columns=df.columns[df.eq(0).mean()>0.9])

In [123]:
df_no0

,also,always,back,better,bread,clean,employees,even,fast,food,...,sandwiches,service,staff,sub,subs,subway,time,ve,way,would
0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.044984,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.066718,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.059047,0.095114,...,0.000000,0.048198,0.00000,0.000000,0.000000,0.041910,0.000000,0.000000,0.068365,0.000000
2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.075555,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.06646,0.072303,0.000000,0.000000,0.000000,0.000000,0.081921,0.000000
4,0.000000,0.0,0.0,0.0,0.133672,0.000000,0.000000,0.0,0.000000,0.000000,...,0.121718,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,0.000000,0.0,0.0,0.0,0.000000,0.043972,0.049797,0.0,0.043621,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.039314,0.088420,0.000000,0.000000
830,0.000000,0.0,0.0,0.0,0.079178,0.000000,0.000000,0.0,0.000000,0.000000,...,0.144195,0.000000,0.00000,0.000000,0.000000,0.052515,0.000000,0.074988,0.000000,0.000000
831,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.056330,0.000000,0.00000,0.059071,0.000000,0.082061,0.052099,0.000000,0.000000,0.000000
832,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.055683,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.062309,0.000000,0.000000,0.070261


In [124]:
df_no0.columns

Index(['also', 'always', 'back', 'better', 'bread', 'clean', 'employees',
       'even', 'fast', 'food', 'fresh', 'friendly', 'get', 'go', 'good', 'got',
       'great', 'like', 'location', 'love', 'lunch', 'make', 'never', 'one',
       'order', 'ordered', 'people', 'place', 'really', 'sandwich',
       'sandwiches', 'service', 'staff', 'sub', 'subs', 'subway', 'time', 've',
       'way', 'would'],
      dtype='object')

MLR with df_no0 data as x features:

In [125]:
x = df_no0
y = df_review['stars_x']

In [126]:
x, y = np.array(x), np.array(y)

In [127]:
model = LinearRegression().fit(x, y)

In [128]:
r_sq = model.score(x, y)

In [129]:
print(f"intercept: {model.intercept_}")

intercept: 2.738949396383648


In [131]:
print(f"coefficients: {model.coef_}")
df_no0.columns

coefficients: [ 2.54399834  0.10055339 -0.91781998  3.28821594 -1.47221783  1.07264982
  0.12663893  1.28546935  0.44426507 -0.36295139  1.10810419  1.99867756
  0.06306843  0.84905136  1.77691658 -1.03166284  1.72480452 -0.25791568
 -1.04795867  0.99831812  0.62785901 -0.86776899 -2.20898492 -0.25874318
 -1.62865826 -0.38985839  0.08884171  1.05138595 -1.86955455  0.59491393
  1.54527822 -0.44409716  0.63158148  2.12927961  1.99192723 -2.25885983
 -0.06306885  0.33204643  0.81002823 -0.90876077]


Index(['also', 'always', 'back', 'better', 'bread', 'clean', 'employees',
       'even', 'fast', 'food', 'fresh', 'friendly', 'get', 'go', 'good', 'got',
       'great', 'like', 'location', 'love', 'lunch', 'make', 'never', 'one',
       'order', 'ordered', 'people', 'place', 'really', 'sandwich',
       'sandwiches', 'service', 'staff', 'sub', 'subs', 'subway', 'time', 've',
       'way', 'would'],
      dtype='object')

MLR with all 30,000+ unigrams & bigrams:

In [104]:
x = df
y = df_review['stars_x']

In [105]:
x, y = np.array(x), np.array(y)

In [109]:
model = LinearRegression().fit(x, y)

In [110]:
r_sq = model.score(x, y)
r_sq

1.0

In [111]:
print(f"intercept: {model.intercept_}")

intercept: 2.824132603377369


In [112]:
print(f"coefficients: {model.coef_}")

coefficients: [-0.13376532 -0.09220681 -0.15799626 ...  0.12778548  0.1016731
  0.10174769]


In [118]:
max(model.coef_)

0.991794369709788